In [1]:
import glob
import pandas as pd
import pandasql as ps
import numpy as np
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
data_path = '../data/projections/'
final_avg_file = 'final_averages_jaylee.xlsx'
hash_bball_file = 'hashtag_bball_projections.xlsx'

In [40]:
# SCRAPING DRAFT RESULTS
id = 8075
league_id = id

In [2]:
# METHODS

def calculate_zscores(df):
    df_final = df.copy()
    punt_cats = ['fg%','ft%','3pm','ppg','rpg','apg','spg','bpg','tog']
    
    for col in punt_cats:
        z_score = (df_final[col]-df_final[col].mean()) / df_final[col].std()
        if col == 'tog':
            df_final[col+'_Z'] = -1 * z_score.round(2)
        elif col!='tog':
            df_final[col + '_Z'] = z_score.round(2)
        else:
            pass
        
    df_final = _calculate_total_value(df_final)

    return df_final


def punt_cats(df,punt_list=[]):
    df_final = df.copy()
    
    if not punt_list:
        pass
    else:        
        for cat in punt_list:
            df_final.drop(columns=[cat,cat+'_Z'],inplace=True)
            if cat == 'fg%':
                df_final.drop(columns=['fg','fga'],inplace=True)
            elif cat == 'ft%':
                df_final.drop(columns=['ft','fta'],inplace=True)
            else:
                continue

    df_final = _calculate_total_value(df_final)

    return df_final


def _calculate_total_value(df):
    value_cols = []

    for col in df.columns:
        if col.endswith('_Z'):
            value_cols.append(col)

    df['Total'] = df[value_cols].sum(axis=1)
    df = df.sort_values('Total', ascending=False).reset_index(drop=True)

    return df


def _scrape_results(league_id=0000):
    url = "https://basketball.fantasysports.yahoo.com/nba/{0}/draftresults".format(league_id)
    players = []

    html = urlopen(url)
    soup = BeautifulSoup(html)

    for line in soup.find_all('td'):
        if line.a == None:
            continue
        else:
            players.append(line.a.get_text())

    return players


def find_best_available(df,league_id=0000):
    """
    Scrape draft results and store the player names being taken and recorded

    Take out row based on player name

    Every time you run the returning DataFrame provides only names of those available

    :param df:
    :return:
    """

    players_drafted = _scrape_results(league_id)

    filter = df['Player'].isin(players_drafted)
    df = df.loc[~filter].head(30).reset_index(drop=True)

    return df


def _clean_player_names(df):
    player_names = []
    camel_case_last_names = ('Gilgeous-Alexander','LaVine','VanVleet','DeRozan','DiVincenzo',
                             'McGee',"O'Neale",'LeVert','Carter-Williams','McCollum')
    for player in df.Player:
        
        player_list = player.split(' ')[:3]
        
        if len(player_list) > 2 and (player_list[2].startswith('Jr.') or player_list[2].startswith('II')):
            
            prefix = player_list[2]
            
            if prefix.startswith('Jr.'):
                split_pre =  re.findall('[A-Z][^A-Z]*', prefix)
                split_pre = re.match('[a-zA-Z_.-]+\w*',split_pre[0])
            elif prefix.startswith('II'):
                split_pre = re.findall('I+', prefix)
                split_pre = re.match('[A-Z]\w*',split_pre[0])
            
            player_list[2] = split_pre[0]
            player_string = ' '.join(player_list)
            player_names.append(player_string)
            
        else:
            
            player_list = player_list[:2]
            last_name = player_list[1]
            
            if last_name.startswith(camel_case_last_names) and last_name not in camel_case_last_names:
                split_last = re.match('([^A-Z]*[A-Z]){2}[a-z]*', last_name)
                split_last = re.match('[a-zA-Z_.-]+\w*',split_last[0])
            elif last_name in camel_case_last_names:
                split_last = [last_name]
                split_last = re.match('[A-Z]\w*',split_last[0])
            else:
                split_last = re.findall('[A-Z][^A-Z]*', last_name)
                split_last = re.match('[A-Z]\w*',split_last[0])
            
            player_list[1] = split_last[0]
            player_string = ' '.join(player_list)
            player_names.append(player_string)
   
    player_col = pd.Series(player_names)
    return player_col


def _separate_percentages(col):
    
    list_percentages = []
    list_made = []
    list_attempts = []
    
    for percentage in col:
        stat = percentage.split('(')
        perc = stat[0]
        m = stat[1].split('/')[0]
        a = stat[1].split('/')[1].split(')')[0]
        list_percentages.append(perc)
        list_made.append(m)
        list_attempts.append(a)
    
    col_percent = pd.Series(list_percentages)
    col_made = pd.Series(list_made)
    col_attempt = pd.Series(list_attempts)
    
    return col_percent, col_made, col_attempt
    

def clean_hashtag_df(df):
    df_final = df.copy()
    df_final.drop(columns=['Pos','Tm','G','Mp','Total'],inplace=True)
    df_final['Player'] = _clean_player_names(df_final)
    df_final['fg%'],df_final['fg'],df_final['fga'] = _separate_percentages(df_final['fg%'])    
    df_final['ft%'],df_final['ft'],df_final['fta'] = _separate_percentages(df_final['ft%'])

    # Re-arrange order of columns
    columns = ['Player','fg%','fg','fga','ft%','ft','fta','3pm','ppg',
               'rpg','apg', 'spg','bpg','tog']
    
    for col in df_final.columns[1:]:
        df_final[col] = pd.to_numeric(df_final[col])

    df_final = df_final[columns]
    
    if len(df_final) < len(df):
        print("Warning! Cleaning data removed player(s)")
    
    return df_final


def _scrape_by_team(name,id):
    url = "https://basketball.fantasysports.yahoo.com/nba/{0}/draftresults".format(league_id)
    players_selected = []

    html = urlopen(url)
    soup = BeautifulSoup(html)
    
    for line in soup.find_all('tr'):
        if line.a==None:
            continue
        else:
            if line.find_all('td')[2].get_text() == name:
                players_selected.append(line.a.get_text())
            else:
                continue
                
    return players_selected


def track_team_stats(df,name,id):
    
    players_selected = _scrape_by_team(name,id)
    
    team_df = df.loc[df['Player'].isin(players_selected)]
    team_df = team_df.append(team_df.mean(numeric_only=True), ignore_index=True)
    team_df = team_df.replace(np.nan, 'AVERAGES')
    
    if 'fg' in team_df.columns:
        team_df.loc[team_df['Player']=='AVERAGES','fg%']=(team_df['fg']/team_df['fga']).round(2)
    elif 'ft' in team_df.columns:
        team_df.loc[team_df['Player']=='AVERAGES','ft%']=(team_df['ft']/team_df['fta']).round(2)
    
    return team_df.round(2)

def track_team_stats_by_list(df,list_players):
    team_df = df.loc[df['Player'].isin(list_players)]
    team_df = team_df.append(team_df.mean(numeric_only=True), ignore_index=True)
    team_df = team_df.replace(np.nan, 'AVERAGES')
    
    if 'fg' in team_df.columns:
        team_df.loc[team_df['Player']=='AVERAGES','fg%']=(team_df['fg']/team_df['fga']).round(2)
    elif 'ft' in team_df.columns:
        team_df.loc[team_df['Player']=='AVERAGES','ft%']=(team_df['ft']/team_df['fta']).round(2)
    
    return team_df.round(2)

def find_best_cats_available(df, lid, cats=['fg%','ft%','3pm','ppg','rpg','apg','spg','bpg','tog']):
    df_final = df.copy()
    # call scrape method
    players_drafted = _scrape_results(lid)
    filter = df_final['Player'].isin(players_drafted)
    df_final = df_final.loc[~filter].head(30).reset_index(drop=True)
    
    # call punt cats method
    all_cats = ['fg%','ft%','3pm','ppg','rpg','apg','spg','bpg','tog']
    c = set(cats)
    
    punt_cats_list = [x for x in all_cats if x not in c]
    df_final = punt_cats(df_final,punt_cats_list)
    
    return df_final

In [6]:
hash_bball = pd.read_excel(data_path + hash_bball_file, engine='openpyxl')

In [7]:
# Copy data
df = hash_bball.copy()

In [8]:
# Clean data
df = clean_hashtag_df(df)

In [9]:
# Adjust OVERALL Top Rankings 
df_100 = calculate_zscores(df.head(100))
df_150 = calculate_zscores(df.head(150))
df_200 = calculate_zscores(df.head(200))
df_250 = calculate_zscores(df.head(250))
df_300 = calculate_zscores(df.head(300))

In [ ]:
# WORK FLOW

# 1) calculate z-score values with all categories and that is the main data frame
# 2) create a df for players adjusted values based on punting categories

In [11]:
punt_to_fg = punt_cats(df_250,['fg%','tog'])
punt_to_fg_blk = punt_cats(df_300,['fg%','tog','bpg'])
punt_to_fg_reb = punt_cats(df_300,['fg%','tog','rpg'])
punt_to_fg_ast_stl_3pm = punt_cats(df_300,['fg%','tog','apg','spg','3pm'])
punt_none = punt_cats(df_300)

In [60]:
find_best_available(df_250,league_id)

,Player,fg%,fg,fga,ft%,ft,fta,3pm,ppg,rpg,...,fg%_Z,ft%_Z,3pm_Z,ppg_Z,rpg_Z,apg_Z,spg_Z,bpg_Z,tog_Z,Total
0,Derrick White,0.467,4.7,10.0,0.838,3.2,3.8,1.5,14.1,4.0,...,-0.13,0.78,0.05,0.07,-0.50,0.71,0.25,1.03,0.09,2.35
1,Duncan Robinson,0.468,4.5,9.6,0.902,0.9,1.0,3.8,13.7,3.2,...,-0.11,1.56,2.50,-0.00,-0.84,-0.62,-0.56,-0.66,1.07,2.34
2,Al Horford,0.473,5.0,10.5,0.805,1.0,1.2,1.3,12.2,7.2,...,-0.03,0.37,-0.16,-0.27,0.83,0.56,-0.29,0.61,0.58,2.20
3,Mikal Bridges,0.482,3.8,8.0,0.823,1.6,1.9,1.5,10.8,4.5,...,0.12,0.59,0.05,-0.51,-0.29,-0.47,2.14,0.18,0.34,2.15
4,Richaun Holmes,0.628,4.6,7.2,0.776,1.9,2.4,0.0,11.0,7.2,...,2.51,0.02,-1.55,-0.48,0.83,-1.03,-0.29,1.24,0.82,2.07
5,Brandon Clarke,0.597,5.8,9.7,0.766,1.7,2.2,0.4,13.6,6.8,...,2.00,-0.10,-1.12,-0.02,0.66,-0.67,-0.56,1.03,0.70,1.92
6,Norman Powell,0.496,5.6,11.3,0.848,2.5,2.9,2.2,15.8,3.5,...,0.35,0.90,0.79,0.36,-0.71,-0.57,0.79,-0.45,0.34,1.80
7,Jarrett Allen,0.642,4.3,6.7,0.662,2.4,3.6,0.0,11.0,9.6,...,2.74,-1.38,-1.55,-0.48,1.83,-0.77,-1.10,1.66,0.70,1.65
8,Thomas Bryant,0.594,5.5,9.3,0.738,1.8,2.4,0.8,13.6,7.1,...,1.95,-0.45,-0.70,-0.02,0.79,-0.57,-1.10,1.03,0.70,1.63
9,Daniel Theis,0.553,4.0,7.3,0.761,1.4,1.9,0.6,10.1,6.9,...,1.28,-0.17,-0.91,-0.63,0.71,-0.62,-0.56,1.45,0.95,1.50


In [69]:
find_best_available(punt_to_fg,league_id)

,Player,ft%,ft,fta,3pm,ppg,rpg,apg,spg,bpg,ft%_Z,3pm_Z,ppg_Z,rpg_Z,apg_Z,spg_Z,bpg_Z,Total
0,Alec Burks,0.880,2.8,3.2,1.7,13.6,4.7,3.1,1.0,0.3,1.29,0.26,-0.02,-0.21,0.10,0.25,-0.66,1.01
1,Darius Garland,0.879,1.6,1.8,2.3,15.2,2.1,4.4,0.9,0.1,1.28,0.90,0.26,-1.30,0.76,-0.02,-1.08,0.80
2,Kevin Porter Jr.,0.754,2.1,2.7,1.8,13.9,4.3,2.7,1.1,0.4,-0.25,0.37,0.03,-0.38,-0.11,0.52,-0.45,-0.27
3,JJ Redick,0.898,2.8,3.1,3.2,15.8,2.6,2.1,0.3,0.2,1.51,1.86,0.36,-1.09,-0.42,-1.64,-0.87,-0.29
4,Trevor Ariza,0.881,0.7,0.8,1.7,7.2,4.8,1.7,1.2,0.2,1.30,0.26,-1.14,-0.17,-0.62,0.79,-0.87,-0.45
5,Gary Harris,0.820,1.4,1.7,1.4,11.5,3.1,2.2,1.4,0.3,0.56,-0.06,-0.39,-0.88,-0.36,1.33,-0.66,-0.46
6,Maxi Kleber,0.834,1.2,1.5,1.8,10.3,5.6,1.4,0.3,1.2,0.73,0.37,-0.60,0.16,-0.77,-1.64,1.24,-0.51
7,Dillon Brooks,0.788,2.2,2.8,2.0,16.1,3.3,2.1,0.9,0.4,0.16,0.58,0.42,-0.79,-0.42,-0.02,-0.45,-0.52
8,Marc Gasol,0.732,0.8,1.1,1.3,7.7,6.3,3.4,0.8,0.9,-0.52,-0.16,-1.05,0.46,0.25,-0.29,0.61,-0.70
9,Jordan Clarkson,0.866,2.7,3.1,2.1,15.2,2.5,2.5,0.6,0.3,1.12,0.69,0.26,-1.13,-0.21,-0.83,-0.66,-0.76


In [87]:
track_team_stats(punt_to_fg,"Josh's Team",league_id)

,Player,ft%,ft,fta,3pm,ppg,rpg,apg,spg,bpg,ft%_Z,3pm_Z,ppg_Z,rpg_Z,apg_Z,spg_Z,bpg_Z,Total
0,Paul George,0.88,4.50,5.10,3.70,24.00,6.4,3.90,1.70,0.4,1.23,2.39,1.80,0.50,0.51,2.14,-0.45,8.12
1,Trae Young,0.87,7.80,9.00,3.40,26.80,3.5,9.60,1.00,0.1,1.12,2.07,2.29,-0.71,3.42,0.25,-1.08,7.36
2,Fred VanVleet,0.84,3.20,3.80,2.80,18.20,3.8,6.80,1.90,0.3,0.78,1.43,0.78,-0.59,1.99,2.68,-0.66,6.41
3,D'Angelo Russell,0.82,3.20,3.90,3.40,20.90,3.6,6.10,1.20,0.3,0.61,2.07,1.26,-0.67,1.63,0.79,-0.66,5.03
4,Robert Covington,0.82,1.20,1.50,2.50,12.30,7.0,1.40,1.60,1.2,0.50,1.11,-0.25,0.75,-0.77,1.87,1.24,4.45
5,Marcus Smart,0.84,2.10,2.50,2.30,13.00,3.8,4.60,1.60,0.5,0.76,0.90,-0.13,-0.59,0.86,1.87,-0.24,3.43
6,Elfrid Payton,0.69,1.60,2.30,0.60,11.10,5.1,7.70,1.80,0.5,-1.08,-0.91,-0.46,-0.04,2.45,2.41,-0.24,2.13
7,Kevin Love,0.86,3.00,3.50,2.40,16.00,7.9,2.70,0.50,0.3,1.09,1.01,0.40,1.12,-0.11,-1.10,-0.66,1.75
8,Duncan Robinson,0.90,0.90,1.00,3.80,13.70,3.2,1.70,0.70,0.3,1.56,2.50,-0.00,-0.84,-0.62,-0.56,-0.66,1.38
9,Terrence Ross,0.85,2.30,2.70,2.60,14.70,3.2,1.20,1.10,0.3,0.95,1.22,0.17,-0.84,-0.88,0.52,-0.66,0.48


In [86]:
track_team_stats(punt_to_fg,"The Kang Center",league_id)

,Player,ft%,ft,fta,3pm,ppg,rpg,apg,spg,bpg,ft%_Z,3pm_Z,ppg_Z,rpg_Z,apg_Z,spg_Z,bpg_Z,Total
0,Luka Doncic,0.77,7.20,9.40,2.90,29.40,9.50,9.10,1.10,0.30,-0.08,1.54,2.75,1.79,3.17,0.52,-0.66,9.03
1,Russell Westbrook,0.74,4.80,6.40,0.90,25.40,8.40,8.80,1.50,0.40,-0.42,-0.59,2.05,1.33,3.01,1.60,-0.45,6.53
2,Kyle Lowry,0.85,4.80,5.70,2.60,18.10,4.60,7.20,1.40,0.40,0.90,1.22,0.77,-0.25,2.20,1.33,-0.45,5.72
3,Jaren Jackson Jr.,0.72,2.50,3.50,2.80,18.80,5.00,1.50,0.80,1.80,-0.61,1.43,0.89,-0.09,-0.72,-0.29,2.50,3.11
4,Ja Morant,0.78,3.90,5.00,1.30,19.70,4.10,8.00,1.10,0.30,0.10,-0.16,1.05,-0.46,2.60,0.52,-0.66,2.99
5,Devonte' Graham,0.82,2.50,3.00,3.00,15.50,2.90,5.80,1.10,0.20,0.62,1.65,0.31,-0.96,1.48,0.52,-0.87,2.75
6,DeMar DeRozan,0.85,5.50,6.50,0.10,21.00,5.40,5.30,1.00,0.30,0.91,-1.44,1.27,0.08,1.22,0.25,-0.66,1.63
7,Lou Williams,0.86,4.30,4.90,1.60,17.50,2.90,5.30,0.70,0.20,1.09,0.16,0.66,-0.96,1.22,-0.56,-0.87,0.74
8,Blake Griffin,0.76,4.30,5.60,1.60,16.30,7.30,3.60,0.50,0.40,-0.15,0.16,0.45,0.87,0.35,-1.10,-0.45,0.13
9,Danilo Gallinari,0.90,3.50,3.90,2.30,14.30,4.00,1.50,0.60,0.10,1.49,0.90,0.10,-0.50,-0.72,-0.83,-1.08,-0.64


In [80]:
track_team_stats(df_250,"Josh's Team",league_id)[z_cats]

,Player,fg%_Z,ft%_Z,3pm_Z,ppg_Z,rpg_Z,apg_Z,spg_Z,bpg_Z,tog_Z
0,Paul George,-0.70,1.23,2.39,1.80,0.50,0.51,2.14,-0.45,-1.37
1,Fred VanVleet,-1.03,0.78,1.43,0.78,-0.59,1.99,2.68,-0.66,-0.64
2,Trae Young,-0.52,1.12,2.07,2.29,-0.71,3.42,0.25,-1.08,-2.59
3,Robert Covington,-0.83,0.50,1.11,-0.25,0.75,-0.77,1.87,1.24,-0.03
4,D'Angelo Russell,-0.75,0.61,2.07,1.26,-0.67,1.63,0.79,-0.66,-1.37
5,Duncan Robinson,-0.11,1.56,2.50,-0.00,-0.84,-0.62,-0.56,-0.66,1.07
6,Richaun Holmes,2.51,0.02,-1.55,-0.48,0.83,-1.03,-0.29,1.24,0.82
7,Marcus Smart,-1.57,0.76,0.90,-0.13,-0.59,0.86,1.87,-0.24,-0.15
8,Wendell Carter Jr.,0.54,-0.40,-1.33,-0.25,1.96,-0.83,-0.02,1.24,-0.03
9,Maxi Kleber,0.26,0.73,0.37,-0.60,0.16,-0.77,-1.64,1.24,1.07


In [78]:
z_cats = ['Player','fg%_Z','ft%_Z','3pm_Z','ppg_Z','rpg_Z','apg_Z','spg_Z','bpg_Z','tog_Z']

In [81]:
def calculate_net_value(df):
    df_final = df.copy()
    
    df_final = df_final[:-1]
    df_f
    
    return df

In [84]:
jaylee[:-1]

,Player,fg%,fg,fga,ft%,ft,fta,3pm,ppg,rpg,...,fg%_Z,ft%_Z,3pm_Z,ppg_Z,rpg_Z,apg_Z,spg_Z,bpg_Z,tog_Z,Total
0,Paul George,0.43,7.9,18.3,0.88,4.5,5.1,3.7,24.0,6.4,...,-0.70,1.23,2.39,1.80,0.50,0.51,2.14,-0.45,-1.37,6.05
1,Fred VanVleet,0.41,6.1,14.8,0.84,3.2,3.8,2.8,18.2,3.8,...,-1.03,0.78,1.43,0.78,-0.59,1.99,2.68,-0.66,-0.64,4.74
2,Trae Young,0.44,7.8,17.6,0.87,7.8,9.0,3.4,26.8,3.5,...,-0.52,1.12,2.07,2.29,-0.71,3.42,0.25,-1.08,-2.59,4.25
3,Robert Covington,0.42,4.3,10.2,0.82,1.2,1.5,2.5,12.3,7.0,...,-0.83,0.50,1.11,-0.25,0.75,-0.77,1.87,1.24,-0.03,3.59
4,D'Angelo Russell,0.43,7.1,16.6,0.82,3.2,3.9,3.4,20.9,3.6,...,-0.75,0.61,2.07,1.26,-0.67,1.63,0.79,-0.66,-1.37,2.91
5,Duncan Robinson,0.47,4.5,9.6,0.90,0.9,1.0,3.8,13.7,3.2,...,-0.11,1.56,2.50,-0.00,-0.84,-0.62,-0.56,-0.66,1.07,2.34
6,Richaun Holmes,0.63,4.6,7.2,0.78,1.9,2.4,0.0,11.0,7.2,...,2.51,0.02,-1.55,-0.48,0.83,-1.03,-0.29,1.24,0.82,2.07
7,Marcus Smart,0.38,4.3,11.4,0.84,2.1,2.5,2.3,13.0,3.8,...,-1.57,0.76,0.90,-0.13,-0.59,0.86,1.87,-0.24,-0.15,1.71
8,Wendell Carter Jr.,0.51,4.6,9.0,0.74,2.9,3.9,0.2,12.3,9.9,...,0.54,-0.40,-1.33,-0.25,1.96,-0.83,-0.02,1.24,-0.03,0.88
9,Maxi Kleber,0.49,3.7,7.4,0.83,1.2,1.5,1.8,10.3,5.6,...,0.26,0.73,0.37,-0.60,0.16,-0.77,-1.64,1.24,1.07,0.82
